# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet','averaged_perceptron_tagger'])

# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from operator import itemgetter
import joblib


from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.neighbors import KNeighborsClassifier







[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nadine.Puetzer1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Nadine.Puetzer1\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Nadine.Puetzer1\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [7]:
# load data from database
database_filepath = "disaster_response_pipeline_project/data/DisasterResponse.db"
engine = create_engine('sqlite:///'+database_filepath)
df = pd.read_sql_table('Clean_Disaster_Data', engine)  
#print(df.head())
X = df.message
print(X[0])
Y = df[df.columns[4:]]
#print(Y.iloc[0])
#Y = df[df.columns[4]]
print(Y.iloc[[0]])

Weather update - a cold front from Cuba that could pass over Haiti
   related  request  offer  aid_related  medical_help  medical_products  \
0        1        0      0            0             0                 0   

   search_and_rescue  security  military  child_alone  ...  aid_centers  \
0                  0         0         0            0  ...            0   

   other_infrastructure  weather_related  floods  storm  fire  earthquake  \
0                     0                0       0      0     0           0   

   cold  other_weather  direct_report  
0     0              0              0  

[1 rows x 36 columns]


### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
pipeline =  Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=0)

# train classifier
pipeline.fit(X_train,y_train)

# predict on test data
y_pred = pipeline.predict(X_test)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [34]:
#for col in range(0,len(Y.columns)):
#    report = classification_report(y_test.values[col], y_pred[col],output_dict=True)
#    print("Category: {}, Accuracy: {:0.2f}, Precision: {:0.2f}, Recall: {:0.2f}\n".format(Y.columns[col],report['accuracy'], report['macro avg']['precision'], report['macro avg']['recall']))
    
model_eval = pd.DataFrame(columns=['target_category','accuracy','precision','recall'])
category_names = Y.columns

for col in range(0,len(category_names)):

    report = classification_report(y_test.values[col], y_pred[col],output_dict=True)
    #print("Category: {}, Accuracy: {:0.2f}, Precision: {:0.2f}, Recall: {:0.2f}".format(category_names[col],report['accuracy'], report['macro avg']['precision'], report['macro avg']['recall']))
    result_dict = {"target_category":category_names[col],
                "accuracy":report['accuracy'], 
                "precision": report['macro avg']['precision'],
                "recall": report['macro avg']['recall']}
    result = pd.DataFrame([result_dict])
    model_eval = model_eval.append(result)


print("Model Accuracy: ",model_eval.accuracy.mean(),"(+/- ", model_eval.accuracy.std(),')\n '
      "Model Precision: ",model_eval.precision.mean(),"(+/- ", model_eval.precision.std(),')\n ' 
      "Model Recall: ",model_eval.recall.mean(),"(+/- ", model_eval.recall.std(),')' )

Model Accuracy:  0.9560185185185183 (+/-  0.046658634985336685 )
 Model Precision:  0.8057770618294022 (+/-  0.219582336902623 )
 Model Recall:  0.7431940937149268 (+/-  0.21748133775707618 )


c:\Users\Nadine.Puetzer1\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nadine.Puetzer1\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nadine.Puetzer1\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nadine.Puetzer1\Anaconda3\lib\site-packa

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
'''
Default parameters for estimator MultiOutputClassifier(estimator=RandomForestClassifier
(bootstrap=True,
ccp_alpha=0.0,
class_weight=None,
criterion='gini',
max_depth=None,
max_features='auto',
max_leaf_nodes=None,
max_samples=None,
min_impurity_decrease=0.0,
min_impurity_split=None,
min_samples_leaf=1,
min_samples_split=2,
min_weight_fraction_leaf=0.0,
n_estimators=100,
n_jobs=None,
oob_score=False,
random_state=None,
verbose=0,
warm_start=False))'''


# Create the grid
parameters = {
        'clf__estimator__n_estimators': [50, 100, 200],
#       'clf__estimator__max_depth': [10, 20, 30, 40, 50],
#       'estimator__max_depth': [None, 10, 20, 30, 40, 50],
#       'estimator__max_features': ['auto', 'sqrt', 'log2'],
#       'clf__estimator__min_samples_leaf': [1, 2, 4],
#       'clf__estimator__bootstrap': [True, False],
        'clf__estimator__min_samples_split': [2, 3, 4]

    }    


# Define GridSearchCV, fit model and print best parameters
cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, refit=True, n_jobs=-1)
cv.fit(X_train, y_train)
print(cv.best_params_)

# Prediction with tuned model
y_pred_cv = cv.predict(X_test)


{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 100}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [3]:
for col in range(0,len(Y.columns)):
    report = classification_report(y_test.values[col], y_pred_cv[col],output_dict=True)
    print("Category: {}, Accuracy: {:0.2f}, Precision: {:0.2f}, Recall: {:0.2f}".format(Y.columns[col],report['accuracy'], report['macro avg']['precision'], report['macro avg']['recall']))
    

NameError: name 'Y' is not defined

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [11]:

#additional Features
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)


#pipeline parameters
parameters_2 = [
    {
        'clf': [MultiOutputClassifier(RandomForestClassifier())], 
#        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#        'features__text_pipeline__tfidf__estimator__stop_words': ['english', None],
        'clf__estimator__n_estimators': [50, 100, 200],
        'clf__estimator__min_samples_split': [2, 3, 4]
    },
    {
        'clf': [MultiOutputClassifier(KNeighborsClassifier())],
#        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#        'features__text_pipeline__tfidf__estimator__stop_words': ['english', None],
        'clf__estimator__n_neighbors': list(range(1, 31))
    }
]

#evaluating multiple classifiers based on pipeline parameters
result=[]

for params in parameters_2:

    #classifier
    clf = params['clf'][0]

    #getting arguments by popping out classifier
    params.pop('clf')

    #pipeline
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', clf)
    ])

    #cross validation using GridSearchCV
    cv = GridSearchCV(pipeline, param_grid=params, cv=2, refit=True, n_jobs=-1)
    cv.fit(X_train, y_train)

    #storing result
    result.append\
    (
        {
            'grid': cv,
            'classifier': cv.best_estimator_,
            'best score': cv.best_score_,
            'best params': cv.best_params_,
            'cv': cv.cv
        }
    )

#sorting result by best score
result = sorted(result, key=itemgetter('best score'),reverse=True)

### 9. Export your model as a pickle file

In [12]:

#saving best classifier
model = result[0]['grid']
joblib.dump(model, 'classifier.pickle')

['classifier.pickle']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [13]:
import sys
import nltk
nltk.download(['punkt', 'wordnet','averaged_perceptron_tagger'])

# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from operator import itemgetter
import joblib


from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.neighbors import KNeighborsClassifier




def load_data(database_filepath):
    #load data from database
    engine = create_engine(database_filepath)
    df = pd.read_sql_table('Clean_Disaster_Data', engine)  

    #define X,Y and category_names
    X = df.message
    Y = df[df.columns[4:]]
    category_names = Y.columns
    
    return X,Y,category_names


def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


#additional Features
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)
    

def build_model():
    
    #pipeline parameters
    parameters = [
        {
            'clf': [MultiOutputClassifier(RandomForestClassifier())], 
    #        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
    #        'features__text_pipeline__tfidf__estimator__stop_words': ['english', None],
            'clf__estimator__n_estimators': [50, 100, 200],
            'clf__estimator__min_samples_split': [2, 3, 4]
        },
        {
            'clf': [MultiOutputClassifier(KNeighborsClassifier())],
    #        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
    #        'features__text_pipeline__tfidf__estimator__stop_words': ['english', None],
            'clf__estimator__n_neighbors': list(range(1, 31))
        }
    ]

    #evaluating multiple classifiers based on pipeline parameters
    result=[]

    for params in parameters:

        #classifier
        clf = params['clf'][0]

        #getting arguments by popping out classifier
        params.pop('clf')

        #pipeline
        pipeline = Pipeline([
            ('features', FeatureUnion([

                ('text_pipeline', Pipeline([
                    ('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer())
                ])),

                ('starting_verb', StartingVerbExtractor())
            ])),

            ('clf', clf)
        ])

        #cross validation using GridSearchCV
        cv = GridSearchCV(pipeline, param_grid=params, cv=2, refit=True, n_jobs=-1)
        cv.fit(X_train, y_train)

        #storing result
        result.append\
        (
            {
                'grid': cv,
                'classifier': cv.best_estimator_,
                'best score': cv.best_score_,
                'best params': cv.best_params_,
                'cv': cv.cv
            }
        )

    #sorting result by best score
    result = sorted(result, key=itemgetter('best score'),reverse=True)
    model = result[0]['grid']

    return model


def evaluate_model(model, X_test, Y_test, category_names):
    #Predictions
    Y_pred = model.predict(X_test)

    #Model evaluation
    model_eval = pd.DataFrame(columns=['target_category','accuracy','precision','recall'])

    for col in range(0,len(category_names)):
        report = classification_report(Y_test.values[col], Y_pred[col],output_dict=True)
        #print("Category: {}, Accuracy: {:0.2f}, Precision: {:0.2f}, Recall: {:0.2f}".format(category_names[col],report['accuracy'], report['macro avg']['precision'], report['macro avg']['recall']))
        result = pd.DataFrame({"target_category":category_names[col],
                    "accuracy":report['accuracy'], 
                    "precision": report['macro avg']['precision'],
                    "recall": report['macro avg']['recall']
                    })
        model_eval.append(result)
        print('Model Accuracy: ',model_eval['accuracy'].mean(),'(+/- ', model_eval['accuracy'].std(),')/n ' /
              'Model Precision: ',model_eval['precision'].mean(),'(+/- ', model_eval['precision'].std(),')/n ' /
              'Model Recall: ',model_eval['recall'].mean(),'(+/- ', model_eval['recall'].std(),')' )
              


def save_model(model, model_filepath):
    #saving best classifier as pickle
    joblib.dump(model, model_filepath)


def main():
    if len(sys.argv) == 3:
        database_filepath, model_filepath = sys.argv[1:]
        print('Loading data...\n    DATABASE: {}'.format(database_filepath))
        X, Y, category_names = load_data(database_filepath)
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
        
        print('Building model...')
        model = build_model()
        
        print('Training model...')
        model.fit(X_train, Y_train)
        
        print('Evaluating model...')
        evaluate_model(model, X_test, Y_test, category_names)

        print('Saving model...\n    MODEL: {}'.format(model_filepath))
        save_model(model, model_filepath)

        print('Trained model saved!')

    else:
        print('Please provide the filepath of the disaster messages database '\
              'as the first argument and the filepath of the pickle file to '\
              'save the model to as the second argument. \n\nExample: python '\
              'train_classifier.py ../data/DisasterResponse.db classifier.pkl')


if __name__ == '__main__':
    main()

Please provide the filepath of the disaster messages database as the first argument and the filepath of the pickle file to save the model to as the second argument. 

Example: python train_classifier.py ../data/DisasterResponse.db classifier.pkl


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nadine.Puetzer1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Nadine.Puetzer1\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Nadine.Puetzer1\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
